Process Data Train

In [ ]:
!python2 code/preprocess.py dataset/train-processed.csv

Processing 100000/100000
Saved processed tweets to: dataset/train-processed-processed.csv


In [ ]:
!python2 code/preprocess.py dataset/test-processed.csv

Traceback (most recent call last):
  File "code/preprocess.py", line 107, in <module>
    preprocess_csv(csv_file_name, processed_file_name, test_file=False)
  File "code/preprocess.py", line 81, in preprocess_csv
    positive = int(line[:line.find(',')])
ValueError: invalid literal for int() with base 10: 'is so sad for my apl friend'


In [ ]:
!python2 code/stats.py dataset/train-processed-processed.csv

Processing 100000/100000
Calculating frequency distribution
Saved uni-frequency distribution to dataset/train-processed-processed-freqdist.pkl
Saved bi-frequency distribution to dataset/train-processed-processed-freqdist-bi.pkl

[Analysis Statistics]
Tweets => Total: 100000, Positive: 56462, Negative: 43538
User Mentions => Total: 0, Avg: 0.0000, Max: 0
URLs => Total: 0, Avg: 0.0000, Max: 0
Emojis => Total: 0, Positive: 0, Negative: 0, Avg: 0.0000, Max: 0
Words => Total: 1283269, Unique: 50361, Avg: 12.8327, Max: 41, Min: 0
Bigrams => Total: 1183437, Unique: 392543, Avg: 11.8344


In [ ]:
!python2 code/stats.py dataset/test-processed-processed.csv


Calculating frequency distribution
Saved uni-frequency distribution to dataset/test-processed-processed-freqdist.pkl
Saved bi-frequency distribution to dataset/test-processed-processed-freqdist-bi.pkl

[Analysis Statistics]
Tweets => Total: 0, Positive: 0, Negative: 0
Traceback (most recent call last):
  File "code/stats.py", line 106, in <module>
    print 'User Mentions => Total: %d, Avg: %.4f, Max: %d' % (num_mentions, num_mentions / float(num_tweets), max_mentions)
ZeroDivisionError: float division by zero


In [ ]:
!python2 code/baseline.py TRAIN = True

Correct = 65.32%


In [ ]:
!python2 code/naivebayes.py TRAIN = True

Generating feature vectors
Processing 100000/100000

Extracting features & training batches
Processing 1/1

Testing
Processing 1/1
Correct: 7768/10000 = 77.6800 %


In [ ]:
!python3 code/logistic.py TRAIN = True

Generating feature vectors
Processing 100000/100000

Extracting features & training batches
2022-01-23 15:09:29.368081: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
Iteration 94/180, loss:0.6260, acc:0.73202022-01-23 15:09:43.377706: W tensorflow/core/data/root_dataset.cc:163] Optimization loop failed: CANCELLED: Operation was cancelled
Iteration 180/180, loss:0.5898, acc:0.7540
Epoch: 1, val_acc:0.7392
Accuracy improved from 0.0000 to 0.7392, saving model
Iteration 158/180, loss:0.5499, acc:0.76002022-01-23 15:10:15.087472: W tensorflow/core/data/root_dataset.cc:163] Optimization loop failed: CANCELLED: Operation was cancelled
Iteration 169/180, loss:0.5388, acc:0.74402022-01-23 15:10:16.400445: W tensorflow/core/data/root_dataset.cc:163] Optimization loop failed: CANCELLED: Operation was cancelled
Iteration 180/180, loss:0.5398, acc:0.7660
E

In [ ]:
!python2 code/decisiontree.py TRAIN = True

Generating feature vectors
Processing 100000/100000

Extracting features & training batches
Processing 1/1

Testing
Processing 1/1
Correct: 6788/10000 = 67.8800 %


In [ ]:
!python2 code/randomforest.py TRAIN = True

Generating feature vectors
Processing 100000/100000

Extracting features & training batches
Processing 1/1/usr/local/lib/python2.7/dist-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Testing
Processing 1/1
Correct: 7214/10000 = 72.1400 %


In [ ]:
import utils
import random
import numpy as np
from xgboost import XGBClassifier
from scipy.sparse import lil_matrix
from sklearn.feature_extraction.text import TfidfTransformer

# Performs classification using XGBoost.


FREQ_DIST_FILE = 'dataset/train-processed-processed-freqdist.pkl'
BI_FREQ_DIST_FILE = 'dataset/train-processed-processed-freqdist-bi.pkl'
TRAIN_PROCESSED_FILE = 'dataset/train-processed-processed.csv'
TEST_PROCESSED_FILE = 'dataset/test-processed-processed.csv'
TRAIN = True
UNIGRAM_SIZE = 1500
VOCAB_SIZE = UNIGRAM_SIZE
USE_BIGRAMS = True
if USE_BIGRAMS:
    BIGRAM_SIZE = 100
    VOCAB_SIZE = UNIGRAM_SIZE + BIGRAM_SIZE
FEAT_TYPE = 'frequency'


def get_feature_vector(tweet):
    uni_feature_vector = []
    bi_feature_vector = []
    words = tweet.split()
    for i in range(len(words) - 1):
        word = words[i]
        next_word = words[i + 1]
        if unigrams.get(word):
            uni_feature_vector.append(word)
        if USE_BIGRAMS:
            if bigrams.get((word, next_word)):
                bi_feature_vector.append((word, next_word))
    if len(words) >= 1:
        if unigrams.get(words[-1]):
            uni_feature_vector.append(words[-1])
    return uni_feature_vector, bi_feature_vector


def extract_features(tweets, batch_size=500, test_file=True, feat_type='presence'):
    num_batches = int(np.ceil(len(tweets) / float(batch_size)))
    for i in range(num_batches):
        batch = tweets[i * batch_size: (i + 1) * batch_size]
        features = lil_matrix((batch_size, VOCAB_SIZE))
        labels = np.zeros(batch_size)
        for j, tweet in enumerate(batch):
            if test_file:
                tweet_words = tweet[1][0]
                tweet_bigrams = tweet[1][1]
            else:
                tweet_words = tweet[2][0]
                tweet_bigrams = tweet[2][1]
                labels[j] = tweet[1]
            if feat_type == 'presence':
                tweet_words = set(tweet_words)
                tweet_bigrams = set(tweet_bigrams)
            for word in tweet_words:
                idx = unigrams.get(word)
                if idx:
                    features[j, idx] += 1
            if USE_BIGRAMS:
                for bigram in tweet_bigrams:
                    idx = bigrams.get(bigram)
                    if idx:
                        features[j, UNIGRAM_SIZE + idx] += 1
        yield features, labels


def apply_tf_idf(X):
    transformer = TfidfTransformer(smooth_idf=True, sublinear_tf=True, use_idf=True)
    transformer.fit(X)
    return transformer


def process_tweets(csv_file, test_file=True):
    """Returns a list of tuples of type (tweet_id, feature_vector)
            or (tweet_id, sentiment, feature_vector)
    Args:
        csv_file (str): Name of processed csv file generated by preprocess.py
        test_file (bool, optional): If processing test file
    Returns:
        list: Of tuples
    """
    tweets = []
    print ('Generating feature vectors')
    with open(csv_file, 'r') as csv:
        lines = csv.readlines()
        total = len(lines)
        for i, line in enumerate(lines):
            if test_file:
                tweet_id, tweet = line.split(',')
            else:
                tweet_id, sentiment, tweet = line.split(',')
            feature_vector = get_feature_vector(tweet)
            if test_file:
                tweets.append((tweet_id, feature_vector))
            else:
                tweets.append((tweet_id, int(sentiment), feature_vector))
            utils.write_status(i + 1, total)
    print ('\n')
    return tweets


if __name__ == '__main__':
    np.random.seed(20)
    unigrams = utils.top_n_words(FREQ_DIST_FILE, UNIGRAM_SIZE)
    if USE_BIGRAMS:
        bigrams = utils.top_n_bigrams(BI_FREQ_DIST_FILE, BIGRAM_SIZE)
    tweets = process_tweets(TRAIN_PROCESSED_FILE, test_file=False)
    if TRAIN:
        train_tweets, val_tweets = utils.split_data(tweets)
    else:
        random.shuffle(tweets)
        train_tweets = tweets
    del tweets
    print ('Extracting features & training batches')
    clf = XGBClassifier(max_depth=25, silent=False, n_estimators=20)
    batch_size = len(train_tweets)
    i = 1
    n_train_batches = int(np.ceil(len(train_tweets) / float(batch_size)))
    for training_set_X, training_set_y in extract_features(train_tweets, test_file=False, feat_type=FEAT_TYPE, batch_size=batch_size):
        utils.write_status(i, n_train_batches)
        i += 1
        if FEAT_TYPE == 'frequency':
            tfidf = apply_tf_idf(training_set_X)
            training_set_X = tfidf.transform(training_set_X)
        clf.fit(training_set_X, training_set_y)
    print ('\n')
    print ('Testing')
    if TRAIN:
        correct, total = 0, len(val_tweets)
        i = 1
        batch_size = len(val_tweets)
        n_val_batches = int(np.ceil(len(val_tweets) / float(batch_size)))
        for val_set_X, val_set_y in extract_features(val_tweets, test_file=False, feat_type=FEAT_TYPE, batch_size=batch_size):
            if FEAT_TYPE == 'frequency':
                val_set_X = tfidf.transform(val_set_X)
            prediction = clf.predict(val_set_X)
            correct += np.sum(prediction == val_set_y)
            utils.write_status(i, n_val_batches)
            i += 1
        print ('\nCorrect: %d/%d = %.4f %%' % (correct, total, correct * 100. / total))
    else:
        del train_tweets
        test_tweets = process_tweets(TEST_PROCESSED_FILE, test_file=True)
        n_test_batches = int(np.ceil(len(test_tweets) / float(batch_size)))
        predictions = np.array([])
        print ('Predicting batches')
        i = 1
        for test_set_X, _ in extract_features(test_tweets, test_file=True, feat_type=FEAT_TYPE):
            if FEAT_TYPE == 'frequency':
                test_set_X = tfidf.transform(test_set_X)
            prediction = clf.predict(test_set_X)
            predictions = np.concatenate((predictions, prediction))
            utils.write_status(i, n_test_batches)
            i += 1
        predictions = [(str(j), int(predictions[j]))
                       for j in range(len(test_tweets))]
        utils.save_results_to_csv(predictions, 'xgboost.csv')
        print ('\nSaved to xgboost.csv')

Generating feature vectors
Processing 100000/100000

Extracting features & training batches
Processing 1/1

Testing
Processing 1/1
Correct: 7203/10000 = 72.0300 %


In [ ]:
!python2 code/svm.py TRAIN = True

Generating feature vectors
Processing 100000/100000

Extracting features & training batches
Processing 1/1

Testing
Processing 1/1
Correct: 7884/10000 = 78.8400 %


In [ ]:
!python3 code/neuralnet.py TRAIN = True

Generating feature vectors
Processing 100000/100000

Extracting features & training batches
2022-01-23 15:21:28.498612: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
Iteration 180/180, loss:0.5145, acc:0.74602022-01-23 15:21:44.678032: W tensorflow/core/data/root_dataset.cc:163] Optimization loop failed: CANCELLED: Operation was cancelled
2022-01-23 15:21:44.974518: W tensorflow/core/data/root_dataset.cc:163] Optimization loop failed: CANCELLED: Operation was cancelled

Epoch: 1, val_acc:0.7621
Accuracy improved from 0.0000 to 0.7621, saving model
Iteration 180/180, loss:0.4896, acc:0.76802022-01-23 15:22:01.305816: W tensorflow/core/data/root_dataset.cc:163] Optimization loop failed: CANCELLED: Operation was cancelled

Epoch: 2, val_acc:0.7672
Accuracy improved from 0.7621 to 0.7672, saving model
Iteration 180/180, loss:0.4754, acc:0.7860
Epoc

In [ ]:
!wget http://nlp.stanford.edu/data/glove.6B.zip

--2022-01-23 15:22:57--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2022-01-23 15:22:57--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2022-01-23 15:22:57--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [ ]:
!unzip glove*.zip

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


In [ ]:
!python3 code/lstm.py

Looking for GLOVE vectors
Processing 400000/0

Found 31735 words in GLOVE
Generating feature vectors
Processing 100000/100000

2022-01-23 15:28:04.082161: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 40, 200)           18000200  
                                                                 
 dropout (Dropout)           (None, 40, 200)           0         
                                                                 
 lstm (LSTM)                 (None, 128)               168448    
                                                                 
 dense (Dense)               (None, 64)                8256      
                                         

In [ ]:
!python3 code/cnn.py

Looking for GLOVE seeds
Processing 400000/0

Generating feature vectors
Processing 100000/100000

2022-01-23 15:31:28.936191: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
Epoch 1/8
704/704 [==============================] - 27s 27ms/step - loss: 0.5327 - accuracy: 0.7243 - val_loss: 0.4834 - val_accuracy: 0.7675
Epoch 2/8
704/704 [==============================] - 18s 26ms/step - loss: 0.4485 - accuracy: 0.7886 - val_loss: 0.4566 - val_accuracy: 0.7815
Epoch 3/8
704/704 [==============================] - 18s 25ms/step - loss: 0.3966 - accuracy: 0.8198 - val_loss: 0.4642 - val_accuracy: 0.7793
Epoch 4/8
704/704 [==============================] - 18s 26ms/step - loss: 0.3491 - accuracy: 0.8451 - val_loss: 0.4792 - val_accuracy: 0.7730
Epoch 5/8
704/704 [==============================] - 18s 26ms/step - loss: 0.3071 - accuracy: 0.8671 - val_loss: